In [ ]:
def replace_symbol(text):
    temp_text = text.replace("_"," ")
    temp_text = temp_text.replace("-LRB-","")
    temp_text = temp_text.replace("-RRB-","")
    temp_text = temp_text.replace("-LSB-","")
    temp_text = temp_text.replace("-RSB-","")
    temp_text = temp_text.replace("\\","")
    temp_text = temp_text.replace("/","")
    temp_text = temp_text.replace(")","")   
    temp_text = temp_text.replace("(","") 
    return temp_text

In [ ]:
# preprocess
from collections import defaultdict
from collections import Counter
import time

wiki = []
topics = {}
replaced_topics = []
topic_list = []

error_sentence = 0

time_start_out = time.time()
document_num = 109
for k in range(document_num):
    #number = str(i+1)
    #filename = 'wiki-pages-text/wiki-%s.txt' % (number.zfill(3))
    filename = 'wiki-pages-text/wiki-%03d.txt' % (k+1)    
    with open(filename, 'r', encoding='UTF-8') as f:
        content_lines = f.readlines() 
        time_start = time.time()
        print('processing %d' % (k+1))
        for i in range(len(content_lines)):
        #for i in range(10):
            wiki_sentence = content_lines[i].split(" ",2)
            topic = wiki_sentence[0]
            
            if (not wiki_sentence[1].isdigit()):
                error_sentence +=1
                continue
                
            sentence_number = int(wiki_sentence[1])
            if topic not in topics:
                topic_id = len(topics)
                topics[topic] = topic_id
                topic_list.append(topic)
                
                replaced_topic = replace_symbol(topic)
                replaced_topics.append(replaced_topic)
                content = replace_symbol(wiki_sentence[2])
                wiki.append({sentence_number:content})
            else :
                topic_id = topics[topic]
                content = replace_symbol(wiki_sentence[2])
                wiki[topic_id][sentence_number] = content
        time_end = time.time()
        cost_time = time_end - time_start
        print('time cost',cost_time,'s')
time_end_out = time.time()
avg_cosumed_time = (time_end_out-time_start_out)/document_num
print('average time', avg_cosumed_time, 's')
print('error_sentence_count',error_sentence)

In [ ]:
## block for generate inverted index

import os
import lucene
from java.io import File
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.document import Document, Field, FieldType
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, IndexOptions, IndexReader, DirectoryReader
from org.apache.lucene.store import SimpleFSDirectory, FSDirectory
from org.apache.lucene.util import Version
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.queryparser.classic import QueryParser

lucene.initVM()
INDEX_DIR = 'index2/'
store = SimpleFSDirectory(File(INDEX_DIR).toPath())
index_writer = IndexWriter(store, IndexWriterConfig(StandardAnalyzer()))

t1 = FieldType()
t1.setOmitNorms(False)
t1.setStored(True)
t1.setTokenized(True)
t1.setIndexOptions(IndexOptions.DOCS_AND_FREQS)

time_start_generate = time.time()

for n, content in enumerate(replaced_topics):  # read in doc here
    doc = Document()
    doc.add(Field("topic_id", str(n), t1))
    doc.add(Field("topic_content", content, t1))
    index_writer.addDocument(doc)   # Add the document to the index

print("%r docs in index" % index_writer.numDocs())
index_writer.commit()
index_writer.close()
time_end_generate = time.time()
print("take ", time_end_generate - time_start_generate, " s to generate inverted index")



In [ ]:
import json
import spacy

nlp = spacy.load('en_core_web_lg')

testing_set = json.load(open('test-unlabelled.json'))
testing_set_out = json.load(open('test-unlabelled.json'))
#testing_set = json.load(open('devset.json'))
#testing_set_out = json.load(open('devset.json'))
#testing_set = json.load(open('test_set_500.json'))
#testing_set_out = json.load(open('test_set_500.json'))

print(len(testing_set))


In [ ]:
## indentify name entity or Pos and generate the query
## block for searching index and comparing 
searcher = IndexSearcher(DirectoryReader.open(store))

process_counter = 0
time_start_process = time.time()
for key, value in testing_set.items():
    process_counter += 1
    claim = value['claim']
    claim = replace_symbol(claim)
    claim_doc = nlp(claim)
    
    if process_counter % 100 == 0:
        print("have processed:",process_counter)
        time_end_process = time.time()
        print(process_counter/(time_end_process-time_start_process))
    
    query = "" 
    
    for token in claim_doc:
        if token.text[0].isupper() and token.pos_ != "DET":
            query += (token.text + " ")
        elif token.pos_ in {"NOUN",'PROPN','NUM'}:
            if token.pos_ != 'NUM' or token.text.isalnum() :
                query += (token.text + " ")
    
    if(len(query) == 0):
        print("this claim has no query:", claim)
        query = claim
    
    my_query = QueryParser("topic_content", StandardAnalyzer()).parse(query)                                                            # type query here
    MAX = 1
    total_hits = searcher.search(my_query, MAX).scoreDocs
    
    candidate_evidences = []
    for hit in total_hits:
        hit_doc = searcher.doc(hit.doc)
        hit_doc_id = int(hit_doc['topic_id'])
        hit_doc_topic = topic_list[hit_doc_id]
        
        for sentence_num, content in wiki[hit_doc_id].items():

            candidate_evidence = nlp(content)
            
            # some evidence is empty ?
            if candidate_evidence.vector_norm == 0:
                #print(candidate_evidence)
                continue
            
            sim = claim_doc.similarity(candidate_evidence)
            
            if sim > 0.79:
                candidate_evidences.append([hit_doc_topic,int(sentence_num)])
                
    if(len(candidate_evidences) > 0 ):
        testing_set_out[key]['label'] = 'SUPPORTS'
        testing_set_out[key]['evidence'] = candidate_evidences
    else:
        testing_set_out[key]['label'] = 'NOT ENOUGH INFO'
        testing_set_out[key]['evidence'] = candidate_evidences

time_end_process = time.time()
time_cost_process = time_end_process - time_start_process
print("take",time_cost_process," s to process")
print("avg:",time_cost_process/process_counter,"s")
with open('testoutput.json', 'w') as f:
    json.dump(testing_set_out, f, indent=2)